In [25]:
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV, KFold, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import xgboost as xgb
import numpy as np

In [26]:
# Đọc dữ liệu
train_df = pd.read_csv('cityu10d_train_dataset.csv')
test_df = pd.read_csv('cityu10d_test_dataset.csv')

In [27]:
# Loại bỏ các cột không cần thiết
train_df = train_df.drop(['ID', 'ApplicationDate'], axis=1)
test_df = test_df.drop(['ID', 'ApplicationDate'], axis=1)

In [28]:
# Xử lý giá trị thiếu
# Lấy các cột numeric và categorical
numeric_cols = train_df.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = train_df.select_dtypes(include=['object']).columns

In [29]:
# Điền giá trị thiếu cho các cột số (numeric columns)
train_df[numeric_cols] = train_df[numeric_cols].fillna(train_df[numeric_cols].median())

In [30]:
# Kiểm tra và điền giá trị thiếu cho test_df chỉ với các cột có trong test_df
numeric_cols_test = numeric_cols.intersection(test_df.columns)
test_df[numeric_cols_test] = test_df[numeric_cols_test].fillna(test_df[numeric_cols_test].median())

In [31]:
# Điền giá trị thiếu cho các cột phân loại (categorical columns)
for col in categorical_cols:
    train_df[col] = train_df[col].fillna(train_df[col].mode()[0])
    test_df[col] = test_df[col].fillna(test_df[col].mode()[0])

In [ ]:
# Thêm các đặc trưng mới vào dữ liệu
train_df['TotalIncome'] = train_df['AnnualIncome'] + train_df['SavingsAccountBalance'] + train_df['CheckingAccountBalance']
test_df['TotalIncome'] = test_df['AnnualIncome'] + test_df['SavingsAccountBalance'] + test_df['CheckingAccountBalance']

train_df['DebtToIncomeRatio_div_Experience'] = train_df['DebtToIncomeRatio'] / (train_df['Experience'] + 1)
test_df['DebtToIncomeRatio_div_Experience'] = test_df['DebtToIncomeRatio'] / (test_df['Experience'] + 1)

train_df['TotalAssets_div_TotalLiabilities'] = train_df['TotalAssets'] / (train_df['TotalLiabilities'] + 1)
test_df['TotalAssets_div_TotalLiabilities'] = test_df['TotalAssets'] / (test_df['TotalLiabilities'] + 1)

train_df['Savings_to_Income'] = train_df['SavingsAccountBalance'] / (train_df['AnnualIncome'] + 1)
test_df['Savings_to_Income'] = test_df['SavingsAccountBalance'] / (test_df['AnnualIncome'] + 1)

## AnhLT: Bổ sung
train_df['DebtPayment_to_Income'] = train_df['MonthlyDebtPayments'] / (train_df['AnnualIncome'] + 1)
test_df['DebtPayment_to_Income'] = test_df['MonthlyDebtPayments'] / (test_df['AnnualIncome'] + 1)
## AnhLT: Bổ sung
train_df['LoanAmount_to_Income'] = train_df['LoanAmount'] / (train_df['AnnualIncome'] + 1)
test_df['LoanAmount_to_Income'] = test_df['LoanAmount'] / (test_df['AnnualIncome'] + 1)

In [33]:
# Mã hóa One-Hot cho các cột phân loại
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoded_train = encoder.fit_transform(train_df[['EmploymentStatus', 'EducationLevel', 'MaritalStatus', 'HomeOwnershipStatus', 'LoanPurpose']])
encoded_test = encoder.transform(test_df[['EmploymentStatus', 'EducationLevel', 'MaritalStatus', 'HomeOwnershipStatus', 'LoanPurpose']])

encoded_train_df = pd.DataFrame(encoded_train, columns=encoder.get_feature_names_out())
encoded_test_df = pd.DataFrame(encoded_test, columns=encoder.get_feature_names_out())

In [34]:
# Kết hợp các đặc trưng mã hóa One-Hot vào dữ liệu ban đầu
train_df = pd.concat([train_df, encoded_train_df], axis=1).drop(['EmploymentStatus', 'EducationLevel', 'MaritalStatus', 'HomeOwnershipStatus', 'LoanPurpose'], axis=1)
test_df = pd.concat([test_df, encoded_test_df], axis=1).drop(['EmploymentStatus', 'EducationLevel', 'MaritalStatus', 'HomeOwnershipStatus', 'LoanPurpose'], axis=1)

In [35]:
# Tách X_train, y_train
y_train = train_df['RiskScore']
X_train = train_df.drop(['RiskScore'], axis=1)

In [36]:
# Chia dữ liệu thành tập train/validation
X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [37]:
# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_split)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(test_df)

# Xác thực chéo K-fold và tinh chỉnh siêu tham số
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [44]:
## AnhLT: điều chỉnh param_dist_xgb
## Cũ:
# param_dist_xgb = {
#     'n_estimators': [100, 200, 300, 500],
#     'max_depth': [3, 5, 7, 10],
#     'learning_rate': [0.1, 0.05, 0.01],
#     'subsample': [0.8, 0.9, 1.0],
#     'colsample_bytree': [0.7, 0.8, 0.9],
#     'gamma': [0, 0.1, 0.2]
# }
# Mới:
param_dist_xgb = {
    'n_estimators': [500, 1000],
    'max_depth': [5, 7, 9],
    'learning_rate': [0.01, 0.005],
    'subsample': [0.7, 0.8],
    'colsample_bytree': [0.7, 0.8],
    'gamma': [0, 0.1],
    'min_child_weight': [1, 3, 5]
}

model_xgb = xgb.XGBRegressor(objective='reg:squarederror', random_state=42, n_jobs=-1)

# AnhLT: điều chỉnh n_iter từ 100 về 50
random_search_xgb = RandomizedSearchCV(
    estimator=model_xgb,
    param_distributions=param_dist_xgb,
    n_iter=50,
    cv=kf,
    scoring='neg_root_mean_squared_error',
    verbose=2,
    random_state=42
)

In [39]:
# Huấn luyện mô hình với RandomizedSearchCV
random_search_xgb.fit(X_train_scaled, y_train_split)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV] END colsample_bytree=0.7, gamma=0, learning_rate=0.005, max_depth=5, min_child_weight=5, n_estimators=500, subsample=0.8; total time=   0.6s
[CV] END colsample_bytree=0.7, gamma=0, learning_rate=0.005, max_depth=5, min_child_weight=5, n_estimators=500, subsample=0.8; total time=   0.6s
[CV] END colsample_bytree=0.7, gamma=0, learning_rate=0.005, max_depth=5, min_child_weight=5, n_estimators=500, subsample=0.8; total time=   0.7s
[CV] END colsample_bytree=0.7, gamma=0, learning_rate=0.005, max_depth=5, min_child_weight=5, n_estimators=500, subsample=0.8; total time=   0.7s
[CV] END colsample_bytree=0.7, gamma=0, learning_rate=0.005, max_depth=5, min_child_weight=5, n_estimators=500, subsample=0.8; total time=   0.7s
[CV] END colsample_bytree=0.8, gamma=0, learning_rate=0.01, max_depth=7, min_child_weight=1, n_estimators=500, subsample=0.8; total time=   1.6s
[CV] END colsample_bytree=0.8, gamma=0, learning_rate=0.01, max

RandomizedSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, grow_policy=None,
                                          importance_type=Non...
                                          multi_strategy=None,
                                          n_estimators=None, n_jobs=-1,
                                          num_parallel_tree=None,
                                          random_state=42, ...),
                   n_iter=50,
                   param_distributions={'colsample_bytree': [0.7, 0.8],
                                        'gamma': [0, 0.1],
                                        'learning_rate': [0.01, 0.005],
                                        'max_depth': [5, 7, 9],
                                        'min_child_weight': [1, 3, 5],
                                        'n_estimators': [500, 1000],
                                        'subsample': [0.7, 0.8]},
                   random_state=42, scoring='neg_root_mean_squared_error',
                   verbose=2)

In [40]:
# Đánh giá mô hình tốt nhất trên tập validation
best_model_xgb = random_search_xgb.best_estimator_
y_pred_val = best_model_xgb.predict(X_val_scaled)
rmse_val = np.sqrt(mean_squared_error(y_val, y_pred_val))
print(f"Validation RMSE: {rmse_val}")

Validation RMSE: 2.446578024454546


In [41]:
# Dự đoán trên tập test
y_pred_test = best_model_xgb.predict(X_test_scaled)

In [42]:
# Lưu kết quả dự đoán
output_df = pd.DataFrame({'ID': range(18001, 20001), 'RiskScore': y_pred_test})
output_df.to_csv('submission_optimized.csv', index=False)